# UHC Example

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('../data/uhc_data.csv')
df['weights'] = 1.0/df.uhc_variance
df.head()

## Build model

In [ ]:
from sfma import Data, Variable, SplineVariable, SplineSpecs, SplineUniformPrior
from sfma.model import SFMAModel

In [ ]:
data = Data(
    col_obs="uhc",
    col_covs=["physicians"],
    col_weights="weights",
    df=df
)

In [ ]:
spline_priors = [
    SplineUniformPrior(order=1, lb=0.0, ub=np.inf, size=20),
    SplineUniformPrior(order=2, lb=-np.inf, ub=0.0, size=20)
]

In [ ]:
variables = [
    SplineVariable("physicians",
                   spline_specs=SplineSpecs(
                       knots=np.linspace(0.0, 1.0, 4),
                       degree=3,
                       knots_type="rel_domain",
                       r_linear=True
                   ),
                   priors=spline_priors)
]

In [ ]:
model = SFMAModel(data, variables)

In [ ]:
model.fit(outlier_pct=0.1, trim_verbose=True, max_iter=10,
          eta_options={"method": "bounded", "bounds": [0.0, 1.0]})

In [ ]:
model.beta

In [ ]:
model.eta

## Predict model

In [ ]:
df_pred = pd.DataFrame({
    "physicians": np.linspace(df.physicians.min(), df.physicians.max(), 100)
})

In [ ]:
df_pred["pred"] = model.predict(df_pred)

In [ ]:
fig, ax = plt.subplots()
outlier_indices = data.trim_weights == 0.0
ax.scatter(df.physicians, df.uhc, color="gray", edgecolor="none", alpha=0.2)
ax.scatter(df.physicians[outlier_indices], df.uhc[outlier_indices], color="red", marker="x", alpha=0.2)
ax.plot(df_pred.physicians, df_pred.pred, color="red")